<a href="https://colab.research.google.com/github/aarohishaiva/CNN_final-Project/blob/main/NLP_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit pyngrok
!pip install diffusers transformers torch



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 10.5 MB/s eta 0:00:00


In [2]:
%%writefile app.py
import streamlit as st
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from diffusers import StableDiffusionPipeline
import torch

# Load the pretrained models from Hugging Face
# Text Summarization
summarization_model = pipeline("summarization")
# Next Word Prediction / Story Prediction / Chatbot (they use the same text-generation model)
text_generation_model = pipeline("text-generation")
# Sentiment Analysis
sentiment_analysis_model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
sentiment_analysis_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
# Question Answering
question_answering_model = pipeline("question-answering")

# Image Generation using Stable Diffusion
image_generation_model = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4").to("cuda" if torch.cuda.is_available() else "cpu")

# User-friendly front end for task selection and input
st.title("Multifunctional NLP and Image Generation Tool")
st.write("Select a task from the dropdown menu:")
task = st.selectbox("Task", ["Text Summarization", "Next Word Prediction", "Story Prediction", "Chatbot", "Sentiment Analysis", "Question Answering", "Image Generation"])

# Process user inputs and generate outputs using the selected models
if task == "Text Summarization":
    text = st.text_area("Enter the text to be summarized:")
    if st.button("Summarize"):
        summary = summarization_model(text, max_length=100, min_length=30, do_sample=False)
        st.write("Summary:")
        st.write(summary[0]["summary_text"])

elif task == "Next Word Prediction":
    text = st.text_area("Enter the text to predict the next word:")
    if st.button("Predict"):
        next_word = text_generation_model(text, max_length=50, num_return_sequences=1)
        st.write("Next Word:")
        st.write(next_word[0]["generated_text"])

elif task == "Story Prediction":
    text = st.text_area("Enter the text to predict the next part of the story:")
    if st.button("Predict"):
        next_part = text_generation_model(text, max_length=150, num_return_sequences=1)
        st.write("Next Part of the Story:")
        st.write(next_part[0]["generated_text"])

elif task == "Chatbot":
    text = st.text_area("Enter your message to the chatbot:")
    if st.button("Send"):
        response = text_generation_model(text, max_length=100, num_return_sequences=1)
        st.write("Chatbot Response:")
        st.write(response[0]["generated_text"])

elif task == "Sentiment Analysis":
    text = st.text_area("Enter the text to analyze the sentiment:")
    if st.button("Analyze"):
        inputs = sentiment_analysis_tokenizer(text, return_tensors="pt")
        outputs = sentiment_analysis_model(**inputs)
        sentiment = torch.argmax(outputs.logits)
        if sentiment == 0:
            st.write("Sentiment: Negative")
        elif sentiment == 1:
            st.write("Sentiment: Positive")

elif task == "Question Answering":
    question = st.text_area("Enter your question:")
    context = st.text_area("Enter the context:")
    if st.button("Answer"):
        answer = question_answering_model(question=question, context=context)
        st.write("Answer:")
        st.write(answer["answer"])

elif task == "Image Generation":
    prompt = st.text_area("Enter the prompt to generate an image:")
    if st.button("Generate"):
        # Generate an image using Stable Diffusion
        with st.spinner("Generating image..."):
            image = image_generation_model(prompt).images[0]
        st.image(image, caption="Generated Image", use_column_width=True)


Writing app.py


In [3]:
from pyngrok import ngrok

# Set ngrok authentication token
ngrok.set_auth_token("2iT1S8jORWP8AFT2gipGnOs9RQe_325YotmHc3irE4pTUSuSp")

# Start ngrok and Streamlit, specifying the port within a configuration
tunnel = ngrok.connect(8501, "http")  # Use 'http' for the protocol
public_url = tunnel.public_url
print(f"Streamlit app running at: {public_url}")

# Start Streamlit
!streamlit run app.py

Streamlit app running at: https://f888-34-171-51-176.ngrok-free.app



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.171.51.176:8501

2024-09-29 17:21:59.626878: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-29 17:21:59.680378: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-29 17:21:59.696173: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-29 17:21:59.744025: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to u